# Alloy blending problem

The company Steelco has received an order for 500 tonnes of steel to be used in shipbuilding.  The steel must have the following charactersitics:

| Chemical Element | Minimum Grade | Maximum Grade |
|------------------|---------------|---------------|
| Carbon (C)       | 2             | 3             |
| Copper (Cu)      | 0.4           | 0.6           |
| Manganese (Mn)   | 1.2           | 1.65          |

The company has seven different raw materials in stock that may be used for the production of this steel. The following table lists the grades, available amounts and prices for all materials:

| Raw Material | C%     | Cu%    | Mn%    | Availability in t | Cost in $/t  |
|--------------|--------|--------|--------|-------------------|--------------|
| Iron1        | 2.5    | 0      | 1.3    | 400               | 200          |
| Iron2        | 3      | 0      | 0.8    | 300               | 250          |
| Iron3        | 0      | 0.3    | 0      | 600               | 150          |
| Cu1          | 0      | 90     | 0      | 500               | 220          |
| Cu2          | 0      | 96     | 4      | 200               | 240          |
| Al1          | 0      | 0.4    | 1.2    | 300               | 200          |
| Al2          | 0      | 0.6    | 0      | 250               | 165          |

The objective is to determine the composition of the steel that minimizes the production cost.

#### Problem data

In [1]:
using NamedArrays

raw = [:iron1, :iron2, :iron3, :cu1, :cu2, :al1, :al2]

elements = [:C, :Cu, :Mn]

composition = [
    2.5 0 1.3
    3 0 0.8
    0 0.3 0
    0 90 0
    0 96 4
    0 0.4 1.2
    0 0.6 0
]

α = NamedArray(composition, (raw, elements), ("Raw Material", "Element"))

# composition (in percent) of [C, Cu, Mn]

# availability in tonnes
availability = Dict(
:iron1 => 400,
:iron2 => 300,
:iron3 => 600,
:cu1 => 500,
:cu2 => 200,
:al1 => 300,
:al2 => 250)

# You can also do it using the fancy Dict(zip()) syntax

cost = Dict(zip(raw,[200,250,150,220,240,200,165]))

# minimum and maximum grade of [C, Cu, Mn]
MinGrade = Dict(zip(elements,[2, 0.4, 1.2]))
MaxGrade = Dict(zip(elements,[3, 0.6, 1.65]))

d = 500
;

#### Solution

In [2]:
using JuMP, HiGHS

m = Model()
set_optimizer(m,HiGHS.Optimizer)

@variable(m, x[raw] >= 0)   # amount of each raw material to use (in tonnes)
@objective(m, Min, sum(cost[i]*x[i] for i in raw))

@expression(m, production, sum(x[i] for i in raw))
@constraint(m, avail[i in raw], x[i] <= availability[i])
@constraint(m, min_grade[e in elements], sum( α[r,e]*x[r] for r in raw) >= MinGrade[e]*production)
@constraint(m, max_grade[e in elements], sum( α[r,e]*x[r] for r in raw) <= MinGrade[e]*production)

@constraint(m, production >= d)


optimize!(m)
println(value.(x))
println("The cost will be \$", objective_value(m))

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
7 rows, 7 cols, 45 nonzeros
4 rows, 7 cols, 26 nonzeros
4 rows, 7 cols, 26 nonzeros
Presolve : Reductions: rows 4(-10); columns 7(-0); elements 26(-26)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 1(1000) 0s
          3     9.8202027263e+04 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 3
Objective value     :  9.8202027263e+04
HiGHS run time      :          0.01
1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [:iron1, :iron2, :iron3, :cu1, :cu2, :al1, :al2]
And data, a 7-element Vector{Float64}:
 

400.0
   0.0
  37.329605033205205
   0.0
   1.712687871373646
  60.95770709542115
   0.0
The cost will be $

98202.02726319469
